# Finding IP Locations in Mass Quantities

By  default Jupyter will only show you the output of the last command in a cell. I want you to see what each of these commands does, so run the following  cell to change that behavior for this notebook.


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Now let's take a look at how Python can enable you to lookup IP address locations based on a free database that you can download from maxmind.com.


In [ ]:
import geoip2.database
reader = geoip2.database.Reader("/home/student/Desktop/GeoLite2-City_20200310/GeoLite2-City.mmdb")
record = reader.city("145.60.3.100")
record



With these commands, we opened the database and created a "reader" that we can use to query records. Then we can pass any IP Address to "reader.city" and it will retrieve the location if it has it in its database. That information was stored in the variable "record" which we printed above. You can also access individual components in the record using some additional Python syntax.  In this next cell we will access the individual records in the results and product a Google Maps link that you can click on to see where the IP address originated.


In [ ]:
record.continent.names.get("en")
record.country.name
record.subdivisions.most_specific.name
record.location.latitude
record.location.longitude
maps_url = f"http://maps.google.com/maps?q={record.location.latitude:0>3.9f},{record.location.longitude:0>3.9f}&z=15"
print(maps_url)


But we have a problem.  If you ask for an IP address that isn't in the database it will cause the database to "Raise an Exception". This is a crash condition for our programs. If this occurs in a python program rather than our notebook the program stops executing and dies.


In [ ]:
record = reader.city("127.0.1.1")


To prevent these errors from crashing our programs, we typically do this inside of an "Exception Handler".  The exception handler will detect when a crash condition has occurred and gracefully handle the error. In this next section I will create a simply python function that takes two inputs. The first is our "reader" that we can use to query the database. The second is the IP address we want to retrieve. If IP Address does not exist in the database it will ignore the address. After creating this new function called **get_geoip2_record()** we will call it once with a bogus IP address and once with a legitimate one.


In [ ]:
def get_geoip2_record(database, ip_address):
    try:
        record = database.city(ip_address)
    except geoip2.errors.AddressNotFoundError:
        record = None
    return record

get_geoip2_record(reader, "127.0.1.1")
get_geoip2_record(reader, "2.2.2.2")



Because the information is local in a database and not across the internet, we can lookup IP addresses very fast! To get a sense for how fast, let's process the same DNS logs we used last time, but this time we won't limit ourselves to only the first 100 entries. We will do all 16,773 of the IP addresses.


In [ ]:
import pathlib
import re
from collections import defaultdict

log_data = pathlib.Path("./dnslog.txt").read_text()
list_of_addresses = re.findall(r"client (\S+?)#", log_data)
by_state = defaultdict(lambda :[])
print("Processing. Please wait...")
for each_ip in list_of_addresses:
    record = get_geoip2_record(reader, each_ip)
    if record:
        location = f"{record.subdivisions.most_specific.name},{record.country.name}"
        by_state[location].append(each_ip)
print("Done!")


Did you blink? You might have missed it. You just processed 168 times the volume of data we did in the previous page where we had to wait 20 seconds or so for 100 records.  We didn't have to watch the results slowly scroll in like last time. Now look at the results broken down by state and country.


In [ ]:
by_state


As you can see with just a little bit of Python code, we can give some context to all of those anonymous IP addresses we find in logs. Now you can focus your attention on those IP addresses that are coming from the same point of origin. If you are interested in one specific State and Country combination you don't have to scroll through all the data. Instead, you can use the .get() method to retrieve the results for a specific State and Country.


In [ ]:
by_state.get("Maine,United States")